# 一些基本概念

在开始学习Keras之前，我们希望传递一些关于Keras，关于深度学习的基本概念和技术，我们建议新手在使用Keras之前浏览以下本页面提到的内容，这将减少你学习中的困惑。

## 符号计算

Keras的底层库使用Theano、Tensorflow或者CNTK三者之一作为Keras的后端。无论是Theano还是Tensorflow又或是CNTK，都是一个“符号式”的库。

因此，这也使得Keras的编程与传统的Python代码有所差别。笼统的说，符号主义的计算首先定义各种变量，然后建立一个“计算图”，计算图规定了各个变量之间的计算关系。建立好的计算图需要编译以确定其内部细节，然而，此时的计算图还是一个“空壳子”，里面没有任何实际的数据，只有当你把需要运算的输入放进去后，才能在整个模型中形成数据流，从而形成输出值。

就像用管道搭建供水系统，当你在拼水管的时候，里面是没有水的。只有所有的管子都接完了，才能送水。

Keras的模型搭建形式就是这种方法，在你搭建Keras模型完毕后，你的模型就是一个空壳子，只有实际生成可调用的函数后（K.function），输入数据，才会形成真正的数据流。

使用计算图的语言，如Theano，以难以调试而闻名，当Keras的debug进入Theano这个层次时，往往也令人头痛。没有经验的开发者很难直观的感受到计算图到底在干些什么。尽管很让人头痛，但大多数的深度学习框架使用的都是符号计算这一套方法，因为符号计算能够提供关键的计算优化、自动求导等功能。

我们建议你在使用前稍微了解一下Theano或Tensorflow，Bing/Google一下即可。

## 张量

张量，或tensor，是本文档会经常出现的一个词汇，在此稍作解释。

使用这个词汇的目的是为了表述统一，张量可以看作是向量、矩阵的自然推广，我们用张量来表示广泛的数据类型。

规模最小的张量是0阶张量，即标量，也就是一个数。

当我们把一些数有序的排列起来，就形成了1阶张量，也就是一个向量。

如果我们继续把一组向量有序的排列起来，就形成了2阶张量，也就是一个矩阵。

把矩阵摞起来，就是3阶张量，我们可以称为一个立方体，具有3个颜色通道的彩色图片就是一个这样的立方体。

把立方体摞起来，好吧，这次我们真的没有给它起别名了，就叫4阶张量了，不要去试图想象4阶张量是什么样子的，它就是个数学上的概念。

张量的阶数有时候也称为纬度，或者轴，轴这个词翻译自英文axis。譬如一个矩阵\[\[1, 2\], \[3, 4\]\]，是一个2阶张量，有两个纬度和轴，沿着第0个轴（为了与Python的计数方式一致，本文档纬度和轴从0算起）你看到的是\[1, 2\]，\[3, 4\]两个向量，沿着第1个轴你看到的是\[1, 3\]，\[2, 4\]两个向量。

要理解“沿着某个轴”是什么意思，不妨试着运行一下下面的代码：

In [1]:
import numpy as np

a = np.array([[1, 2], [3, 4]])
sum0 = np.sum(a, axis=0)
sum1 = np.sum(a, axis=1)

print('sum0:', sum0)
print('sum1:', sum1)

('sum0:', array([4, 6]))
('sum1:', array([3, 7]))


## data_format

这是一个无可奈何的问题，在如何表示一组彩色图片的问题上，Theano和Tensorflow发生了分歧，‘th’模式，也即Theano模式会把100张RGB三通道的$16 \times 32$（高为16宽为32）彩色图片表示为下面这种形式(100, 3, 16, 32)，Caffe也采取这种方式。第0个维度是样本维，代表样本的数目，第1个维度是通道维，代表颜色通道数。后面两个就是高和宽了。这种theano风格的数据组织方法，称为“channel_first”，即通道维靠前。

而Tensorflow的表达形式是(100, 16, 32, 3)，即把通道维放在了最后，这种数据组织方式称为“channel_last”。

Keras默认的数据组织形式在~/.keras/keras.json中规定，可查看该文件的image_data_format一项查看，也可在代码中通过K.image_data_format()函数返回，请在网络的训练和测试中保持维度顺序一致。

## 函数式模型

函数式模型算是本文档比较原创的词汇了，所以这里要说一下。

在Keras 0.x中，模型其实有两种，一种叫Sequential，称为序贯模型，也就是单输入单输出，一条路通到底，层与层之间只有相邻关系，跨层连接统统没有。这种模型编译速度快，操作上也比较简单。第二种模型称为Graph，即图模型，这个模型支持多输入多输出，层与层之间想怎么连就怎么连，但是编译速度慢，可以看到，Sequential其实是Graph的一个特殊情况。

在Keras 1和Keras 2中，图模型被移除，而增加了“functional model API”，这个东西，更加强调了Sequential是特殊情况这一点。一般的模型就称为Model，然后如果你要用简单的Sequential，那还有一个快捷方式Sequential。

由于functional model API在使用时利用的是“函数式编程”的风格，我们这里将其译为函数式模型。总而言之，只要这个东西接收一个或一些张量作为输入，然后输出的也是一个或一些张量，那不管它是什么，统统都称作“模型”。

## batch

这个概念与Keras无关，老实讲不应该出现在这里的，但是因为它频繁出现，而且不了解这个技术的话看函数说明会很头痛，这里还是简单说一下。

深度学习的优化算法，说白了就是梯度下降。每次的参数更新有两种方式。

第一种，遍历全部数据集算一次损失函数，然后算函数对各个参数的梯度，更新梯度。这种方法每更新一次参数都要把数据集里的所有样本都看一遍，计算量开销大，计算速度慢，不支持在线学习，这称为Batch Gradient Descent，批梯度下降。

另一种，每看一个数据就算一下损失函数，然后求梯度更新参数，这个称为随机梯度下降，Stochastic Gradient Descent。这个方法速度比较快，但是收敛性能不太好，可能在最优点附近晃来晃去，达不到最优点。两次参数的更新也有可能互相抵消，造成目标函数震荡的比较剧烈。

为了克服两种方法的缺点，现在一般采用的是一种折中手段，Mini-batch Gradient Descent，小批的梯度下降，这种方法把数据分为若干个批次，按批来更新参数，这样，一个批中的一组数据共同决定了本次梯度的方向，下降起来就不容易跑偏，减少了随机性。另一方面因为批的样本数与整个数据集相比小了很多，计算量也不是很大。

基本上现在的梯度下降都是基于mini-batch的，所以Keras的模块中经常会出现batch_size，就是这个意思。

顺便说一句，Keras中用的优化器SGD是Stochastic Gradient Descent的缩写，但不代表是一个样本就更新一回，还是基于mini-batch的。

## epochs

真的不是很想解释这个词，但是新手问的还挺多的。简单的说，epochs指的就是训练过程中数据将被“轮”多少次。